Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function

import numpy as np
import tensorflow as tf


First reload the data we generated in `1_notmnist.ipynb`.

Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [ ]:
num_of_transcripts = 199324
num_labels = 5

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, num_of_transcripts)).astype(np.float32)
    # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)


In [ ]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])


In [ ]:
batch_size = 16
num_hidden_nodes = [1024, 256, 128]
input_size = [num_of_transcripts] + num_hidden_nodes
num_layers = len(num_hidden_nodes)

graph = tf.Graph()
with graph.as_default():
    # Input
    tf_train_dataset = tf.placeholder(tf.float32, shape=
    (batch_size, num_of_transcripts))
    tf_train_labels = tf.placeholder(tf.float32, shape=
    (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    global_step = tf.Variable(0)

    # Weights
    weights1 = []
    biases1 = []
    beta1 = []

    for i in range(num_layers):
        weights1.append(tf.Variable(tf.truncated_normal([input_size[i], num_hidden_nodes[i]], \
                                                        stddev=np.sqrt(2.0 / (input_size[i])))))
        biases1.append(tf.Variable(tf.zeros([num_hidden_nodes[i]])))
        beta1.append(1e-3)
    biases2 = tf.Variable(tf.zeros([num_labels]))
    weights2 = tf.Variable(tf.truncated_normal([input_size[-1], num_labels], \
                                               stddev=np.sqrt(2.0 / input_size[-1])))
    beta2 = 1e-3

    # Training Computation
    lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1[0]) + biases1[0])
    drop1 = tf.nn.dropout(lay1_train, 0.5)
    for i in range(1, num_layers):
        print(lay1_train.shape)
        lay1_train = tf.nn.relu(tf.matmul(drop1, weights1[i]) + biases1[i])
        drop1 = tf.nn.dropout(lay1_train, 0.5)
    logits = tf.matmul(drop1, weights2) + biases2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)) \
           + beta2 * tf.nn.l2_loss(weights2)
    for i in range(num_layers):
        loss += beta1[i] * tf.nn.l2_loss(weights1[i])

        # Optimizer
    learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.65, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions
    train_prediction = tf.nn.softmax(logits)

    lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1[0]) + biases1[0])
    for i in range(1, num_layers):
        lay1_valid = tf.nn.relu(tf.matmul(lay1_valid, weights1[i]) + biases1[i])
    logits_valid = tf.matmul(lay1_valid, weights2) + biases2
    valid_prediction = tf.nn.softmax(logits_valid)

    lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1[0]) + biases1[0])
    for i in range(1, num_layers):
        lay1_test = tf.nn.relu(tf.matmul(lay1_test, weights1[i]) + biases1[i])
    logits_test = tf.matmul(lay1_test, weights2) + biases2
    test_prediction = tf.nn.softmax(logits_test)


In [ ]:
num_steps = 3001

accuracy_val = []
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialised')
    for step in range(num_steps):
        offset = (step * batch_size) % \
                 (train_labels.shape[0] - batch_size)

        batch_data = train_dataset[offset:offset + batch_size, :]
        batch_labels = train_labels[offset:offset + batch_size, :]

        feed_dict = {tf_train_dataset: batch_data, \
                     tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, \
                                         train_prediction], feed_dict=feed_dict)
        if step % 500 == 0:
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%% " % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%% " % accuracy(valid_prediction.eval(), valid_labels))

    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
